# Requirements

In [1]:
!pip install transformers datasets sentencepiece evaluate accelerate


In [2]:
!pip install evaluate

In [3]:
!pip install rouge-score

In [10]:
!pip install peft accelerate bitsandbytes sentencepiece


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.5 MB/s eta 0:00:0000:0100:01


In [15]:
!pip install -U bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Libraries

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset
import evaluate
from transformers import BitsAndBytesConfig
import torch
from peft import get_peft_model, LoraConfig, TaskType
import numpy as np
from collections import defaultdict
import numpy as np
import evaluate
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

/home/SSD2/mahmoudreda/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [3]:

# Load only the required languages
xl_sum_en = pd.DataFrame(load_dataset("csebuetnlp/xlsum", "english", split="train"))[:10000]
xl_sum_es = pd.DataFrame(load_dataset("csebuetnlp/xlsum", "spanish", split="train"))[:10000]
xl_sum_zh =  pd.DataFrame(load_dataset("csebuetnlp/xlsum", "chinese_simplified", split="train"))[:10000]



In [4]:
dataset = pd.concat([xl_sum_en, xl_sum_es,xl_sum_zh], ignore_index=True)

In [5]:
train,test = train_test_split(dataset, test_size=0.1, random_state=42)

In [6]:


len(train),len(test)

(27000, 3000)

In [7]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

# Tokenizer

In [8]:

model_checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 512
max_target_length = 128


def preprocess(example):
    inputs = example["text"]
    targets = example["summary"]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train = train.map(preprocess, batched=True, remove_columns=["text", "summary", "title", "url", "id"])
test = test.map(preprocess, batched=True, remove_columns=["text", "summary", "title", "url", "id"])


Map: 100%|██████████| 3000/3000 [00:01<00:00, 2587.52 examples/s]


In [9]:
train

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 27000
})

In [10]:
test

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
})

# Model

In [11]:
"""
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",     # best for QLoRA
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)
"""

'\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_quant_type="nf4",     # best for QLoRA\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_compute_dtype=torch.float16,\n)\n'

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)



# Rouge Metrics

In [ ]:

rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Use the same tokenizer used during training
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Compute BERTScore
    bertscore_result = bertscore.compute(
        predictions=decoded_preds,
        references=decoded_labels,
           lang="multilingual"
    )

    result = {key: round(value * 100, 4) for key, value in result.items()}
    result["gen_len"] = np.mean([len(tokenizer.tokenize(pred)) for pred in decoded_preds])
    result["bertscore_f1"] = round(np.mean(bertscore_result["f1"]) * 100, 4)
    result["bertscore_precision"] = round(np.mean(bertscore_result["precision"]) * 100, 4)
    result["bertscore_recall"] = round(np.mean(bertscore_result["recall"]) * 100, 4)
    return result




In [14]:
# Disable P2P and IB communication
import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"


In [15]:
"""lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # or ["k", "q", "v"] depending on attention blocks
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Should show only a few LoRA layers trainable"""


'lora_config = LoraConfig(\n    r=8,\n    lora_alpha=16,\n    target_modules=["q", "v"],  # or ["k", "q", "v"] depending on attention blocks\n    lora_dropout=0.05,\n    bias="none",\n    task_type=TaskType.SEQ_2_SEQ_LM\n)\n\nmodel = get_peft_model(model, lora_config)\nmodel.print_trainable_parameters()  # Should show only a few LoRA layers trainable'

# Training

In [16]:
from transformers import DataCollatorForSeq2Seq

from transformers import DataCollatorForSeq2Seq
collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [ ]:



training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-multilingual-summarizer-new-new",
    eval_strategy="steps",
    learning_rate=5e-5,
    eval_steps=1000,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="steps",
    save_steps = 1000,
    fp16=True,
    push_to_hub=False,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,            
    metric_for_best_model="rougeL",         
greater_is_better=True,   
    generation_max_length=128,   # ✅ sets generation limits
    generation_num_beams=4       # ✅ sets beam search for better outputs
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator = collator

)


/tmp/ipykernel_1817157/1919534951.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:

trainer.train()




Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bertscore F1,Bertscore Precision,Bertscore Recall
1000,1.273900,1.276647,20.526000,6.054300,14.396000,14.384200,79.094333,69.886000,69.205300,70.702100
2000,1.240600,1.217741,20.707400,6.192200,14.577000,14.549000,63.158667,69.682700,69.701000,69.817900
3000,1.213300,1.179448,20.527300,5.916300,14.131800,14.120500,69.398000,69.468600,69.220500,69.845300
4000,1.014900,1.157119,20.948700,6.357300,14.750900,14.715400,70.332333,70.107100,69.739100,70.624600
5000,0.963100,1.143168,21.277000,6.435500,14.992000,14.960100,75.328000,70.232400,69.581200,71.032400
6000,0.981900,1.134357,21.602800,6.763900,15.187400,15.184500,68.886333,70.151000,69.787100,70.679100
7000,0.859200,1.136339,21.420600,6.572100,15.018000,15.000300,70.765333,70.407400,69.886500,71.098800
8000,0.823300,1.136148,21.465400,6.573200,15.093800,15.095900,67.596000,70.073200,69.746700,70.562600
9000,0.868100,1.122890,21.875600,6.864700,15.453900,15.425200,71.496667,70.394100,69.857700,71.099300
10000,0.858700,1.122647,21.928300,7.008200,15.515900,15.502900,72.574000,70.478600,69.893300,71.228700


/home/SSD2/mahmoudreda/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=10125, training_loss=1.0448691087887612, metrics={'train_runtime': 6248.3017, 'train_samples_per_second': 12.964, 'train_steps_per_second': 1.62, 'total_flos': 8.7767736385536e+16, 'train_loss': 1.0448691087887612, 'epoch': 3.0})

# Save model

In [20]:
trainer.save_model("./continue-finetuned-model")
trainer.tokenizer.save_pretrained("./continue-finetuned-model")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('./continue-finetuned-model/tokenizer_config.json',
 './continue-finetuned-model/special_tokens_map.json',
 './continue-finetuned-model/vocab.json',
 './continue-finetuned-model/merges.txt',
 './continue-finetuned-model/added_tokens.json',
 './continue-finetuned-model/tokenizer.json')

In [ ]:
model.save_pretrained("./bart-multilingual-final")
tokenizer.save_pretrained("./bart-multilingual-final")

# Push to Huggingface

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/home/SSD2/mahmoudreda/bart-multilingual-final"


model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)



/home/SSD2/mahmoudreda/.venv/lib/python3.10/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/home/SSD2/mahmoudreda/bart-multilingual-final"


model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)



/home/ubuntu/anaconda3/envs/illm/lib/python3.12/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [5]:
model.push_to_hub("Mahmoud3899/bart-multilingual-final")
tokenizer.push_to_hub("Mahmoud3899/bart-multilingual-final")


/home/ubuntu/anaconda3/envs/illm/lib/python3.12/site-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mahmoud3899/bart-multilingual-final/commit/4e88fea7f30959f023c2e47df280d6eb2f7cd409', commit_message='Upload tokenizer', commit_description='', oid='4e88fea7f30959f023c2e47df280d6eb2f7cd409', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mahmoud3899/bart-multilingual-final', endpoint='https://huggingface.co', repo_type='model', repo_id='Mahmoud3899/bart-multilingual-final'), pr_revision=None, pr_num=None)

# Evaluation

In [1]:
from transformers import pipeline
summarizer = pipeline("summarization", model="bart-multilingual-final", tokenizer="bart-multilingual-final",device=-1)


/home/SSD2/mahmoudreda/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/SSD2/mahmoudreda/.venv/lib/python3.10/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
Device set to use cpu


In [3]:
import pandas as pd
from datasets import load_dataset
test =  pd.DataFrame(load_dataset("csebuetnlp/xlsum", "english", split="train"))
len(test)

Generating validation split: 100%|██████████| 11535/11535 [00:00<00:00, 205068.06 examples/s]


11535

In [4]:
test["text"][0]

'By Kate DaileyBBC News Earlier this week, Trump posted a photo of himself sitting at a desk at Mar-a-Largo, a permanent marker hovering over a notepad. "Writing my inaugural address at the Winter White House, Mar-a-Lago, three weeks ago. Looking forward to Friday," he tweeted. Trump vows to end \'American carnage\' Trump\'s angry call to arms Full text of Trump\'s inauguration speech It\'s unclear whether the president-elect actually wrote the speech himself, but the content was pure Trump: the same populist message that resonated throughout the primaries and the campaign. "Today, we are not merely transferring power from one administration to another, or from one party to another, but we are transferring power from Washington, DC, and giving it back to you, the people," he said at the beginning of his remarks. For some on Twitter, it bore an eerie similarity to the Batman villain Bane\'s speech in The Dark Night Rises, so much so that someone posted a 10-second mash-up of the two. Bu

In [5]:
test["summary"][0]

"Donald Trump campaigned on becoming a president unlike any Washington has ever seen. With his inauguration speech, he's already set the tone."

In [6]:
summarizer(test["text"][0])

[{'summary_text': 'Donald Trump\'s inaugural speech was a call to arms for Americans to come together and fight for what he described as "the cause" of America\'s problems. It was an unusually bleak speech for a president who campaigned as an outsider, and was met with boos and boos from the crowd that greeted his arrival at the Lincoln Memorial. But what did his supporters make of it?'}]

In [25]:
import pandas as pd
from datasets import load_dataset
test =  pd.DataFrame(load_dataset("csebuetnlp/xlsum", "spanish", split="train"))
len(test)

38110

In [30]:
test["text"][9000]

'El ejército le había dado un ultimátum al presidente Morsi: 48 horas para escuchar las demandas de la gente. No está claro que rol tomará el actual presidente Mohamed Morsi durante este proceso y en el período previo a una elección presidencial libre. La agencia de noticias Reuters ha recibido información similar de fuentes militares, que subrayan que todavía se está discutiendo el plan y que podría cambiar. Las Fuerzas Armadas de Egipto advirtieron el lunes que intervendrían si el presidente Morsi no escuchaba las demandas de la gente dentro de las 48 horas. Esto después de que millones de personas salieran a las calles para exigir la renuncia del presidente.'

In [27]:
test["summary"][9000]

'La directora de ayuda humanitaria de Naciones Unidas, Valerie Amos, advirtió que de no invertir más dinero, el Programa Mundial de Alimentos tendrá que detener sus operaciones en Siria en dos meses.'

In [31]:
summarizer(test["text"][9000])

[{'summary_text': 'os en Egipto informan que el ejército le había dado un ultimátum al presidente Mohamed Morsi: 48 horas para escuchar las demandas de la gente, según fuentes.'}]

In [35]:
import pandas as pd
from datasets import load_dataset
test =  pd.DataFrame(load_dataset("csebuetnlp/xlsum", "chinese_simplified", split="train"))
len(test)

37362

In [48]:
test["text"][0]

'软件巨头微软公司的IE6网络浏览器在美国的用户不到美国互联网用户的1%，该公司为此举行了一个随意的庆祝仪式。 微软公司急于终止使用老一代的网络搜索器，鼓励用户升级使用IE8或9网络浏览器。 与此同时，微软公司的对手谷歌被迫减少在网络宣传他们自己的Chrome网络搜索器。'

In [47]:
test["summary"][0]

'微软公司做蛋糕庆祝该公司的IE6网络浏览器终止使用。'

In [46]:
summarizer(test["text"][0])

[{'summary_text': '网络搜索器（IE6）的微软公司，提出的鼓励用户。'}]

# Human Feedback & cultural/linguistic biases

## English

In [49]:
english_text = "Donald Trump delivered a speech that focused on the struggles of ordinary Americans and promised to return power to the people."
spanish_text = "Donald Trump pronunció un discurso centrado en las luchas de los estadounidenses comunes y prometió devolver el poder al pueblo."
mandarin_text = "唐纳德·特朗普发表演讲，强调普通美国人的困境，并承诺将权力还给人民。"


In [50]:
summary_en = summarizer(english_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
summary_es = summarizer(spanish_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
summary_zh = summarizer(mandarin_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']


Your max_length is set to 60, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 60, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


In [ ]:
#compare English vs Spanish summary
bert_result_en_es = bertscore.compute(predictions=[summary_en], references=[summary_es], lang="multilingual")
bert_result_en_zh = bertscore.compute(predictions=[summary_en], references=[summary_zh], lang="multilingual")


In [52]:
print("\n=== BERTSCORE SIMILARITY ===")
print("EN ↔ ES BERTScore F1:", round(bert_result_en_es['f1'][0] * 100, 2))
print("EN ↔ ZH BERTScore F1:", round(bert_result_en_zh['f1'][0] * 100, 2))


=== BERTSCORE SIMILARITY ===
EN ↔ ES BERTScore F1: 69.08
EN ↔ ZH BERTScore F1: 67.35


## Spanish

In [53]:

english_text = "The Spanish government has announced a new economic plan to reduce youth unemployment. The Economy Minister stated that the goal is to create 250,000 new jobs in the next two years through tax incentives for small and medium-sized businesses. There is also a plan for significant investment in vocational training programs for youth."
spanish_text = "El gobierno español ha anunciado un nuevo plan económico para reducir el desempleo juvenil. El ministro de Economía declaró que el objetivo es crear 250.000 nuevos puestos de trabajo en los próximos dos años mediante incentivos fiscales para las pequeñas y medianas empresas. También se prevé una inversión significativa en programas de formación profesional para jóvenes."
mandarin_text = "西班牙政府宣布了一项新的经济计划，旨在降低青年失业率。经济部长表示，目标是在未来两年内通过对中小企业的税收激励措施创造25万个新的就业岗位。该计划还包括对青年职业培训项目的大规模投资。"

In [54]:
summary_en = summarizer(english_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
summary_es = summarizer(spanish_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
summary_zh = summarizer(mandarin_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']


In [ ]:
#compare English vs Spanish summary
bert_result_es_en = bertscore.compute(predictions=[summary_es], references=[summary_en], lang="multilingual")
bert_result_es_zh = bertscore.compute(predictions=[summary_es], references=[summary_zh], lang="multilingual")


In [56]:
print("\n=== BERTSCORE SIMILARITY ===")
print("ES ↔ EN BERTScore F1:", round(bert_result_es_en['f1'][0] * 100, 2))
print("ES ↔ ZH BERTScore F1:", round(bert_result_es_zh['f1'][0] * 100, 2))


=== BERTSCORE SIMILARITY ===
ES ↔ EN BERTScore F1: 67.3
ES ↔ ZH BERTScore F1: 67.27


## Chinese 

In [57]:

english_text = "China's Cyberspace Administration has recently issued a new regulation requiring all major tech companies to report to regulators before launching new algorithmic services. The measure aims to strengthen oversight of AI and recommendation systems to protect user privacy and prevent information manipulation. This regulation is seen as an important step in China's efforts to enhance internet governance."
spanish_text = "La Administración del Ciberespacio de China ha emitido recientemente una nueva regulación que exige a todas las principales empresas tecnológicas informar a los reguladores antes de lanzar nuevos servicios basados en algoritmos. La medida tiene como objetivo reforzar la supervisión de la inteligencia artificial y los sistemas de recomendación para proteger la privacidad de los usuarios y prevenir la manipulación de la información. Esta regulación se considera un paso importante en los esfuerzos de China por mejorar la gobernanza de Internet."
mandarin_text = "中国国家网信办近日发布了一项新的规定，要求所有大型科技公司在推出新的算法服务前，必须向监管机构报告。该措施旨在加强对人工智能和推荐算法的监管，以保护用户隐私并防止信息操控。这一规定被认为是中国在加强互联网治理方面迈出的重要一步。"

In [58]:
summary_en = summarizer(english_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
summary_es = summarizer(spanish_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
summary_zh = summarizer(mandarin_text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']


In [59]:
bert_result_zh_es = bertscore.compute(predictions=[summary_zh], references=[summary_en], lang="multilingual")
bert_result_eh_en = bertscore.compute(predictions=[summary_zh], references=[summary_es], lang="multilingual")


In [60]:
print("\n=== BERTSCORE SIMILARITY ===")
print("ES ↔ EN BERTScore F1:", round(bert_result_zh_es['f1'][0] * 100, 2))
print("ES ↔ ZH BERTScore F1:", round(bert_result_eh_en['f1'][0] * 100, 2))


=== BERTSCORE SIMILARITY ===
ES ↔ EN BERTScore F1: 66.24
ES ↔ ZH BERTScore F1: 61.66
